# Indicadores técnicos

Para los ejercicios de este notebook, utilizará los datos que se encuentran en datos/sp500.csv

In [2]:
import numpy as np 
#Lee datos
#Columna Adj Close
#Los precios están ordenados del más antiguo (datos[0]) al más reciente (datos[-1])
datos = np.loadtxt('datos/sp500.csv', skiprows=1, usecols=(5), delimiter=',')


# Promedios móviles

Los promedios móviles son uno de los indicadores técnicos más famosos. Como lo señala su nombre, estos indicadores se calculan promediando los valores de una serie de tiempo que se encuentran dentro de una ventana de tiempo en particular.

Para los promedios móviles $M$, se tienen las siguientes señales:

+ Si $P_t > M_{t|w}$ y $P_{t - 1} < M_{t-1 | w}$, entonces **compra en $t$**.
+ Si $P_t < M_{t|w}$ y $P_{t - 1} > M_{t-1 | w}$, entonces **vende en $t$**.

## Promedio móvil simple
Sea $\{P_{t}\} = \{P_1, P_2, \ldots, P_n\}$ una serie de tiempo con $n$ observaciones (la observación más reciente es $P_n$). Sea $w$ el tamaño de la ventana de tiempo. El promedio móvil simple para el periodo $t$ se define como:
$$
SMA_{t|w} = \dfrac{P_t + P_{t-1} + \ldots + P_{t-w + 1} }{w}
$$
para $t \in [w, n]$.

## Promedio móvil exponencial
A diferencia del promedio móvil simple, el promedio móvil exponencial puede capturar el hecho de que la información del pasado no tiene el mismo impacto que la información más reciente.

$$
EMA_{t|w,\lambda} = \left(1 - \lambda \right)EMA_{t-1 | w, \lambda}  + \lambda P_{t}
$$

en donde $0 \leq \lambda \leq 1$ y $t \in [w + 1, n]$. El valor inicial de este promedio usualmente es $EMA_{w|w, \lambda} = SMA_{w|w}$.

### Ejercicios

+ Programe funciones para calcular los dos tipos de promedio móviles anteriores. El resultado debe de ser un numpy array, con la misma longitud de 'datos', conteniendo los valores del indicador. En caso de que no se tenga suficiente información para calcular el valor en un momento $t$, el valor que se debe de asignar es numpy.nan

+ Programe una función que genere las señales de compra y venta para cada periodo $t$. El resultado debe de ser un numpy array de strings que representa la señal para dicho periodo. ('compra', 'venta' o 'espera'). Si el valor del indicador, en el periodo $t$, es numpy.nan entonces se asigna 'espera'.

+ Programe una función que grafique la serie de tiempo original junto con sus promedios móviles.

In [ ]:
#Escriba su código aquí

# Relative Strength Index (RSI)

El RSI es un indicador técnico que se encuentra acotado dentro del intervalo $[0,1]$. Para calcular este indicador, necesitamos primero calcular el indicador $RS$ (Relative Strength) dado por:

$$
RS_{t|w} = \dfrac{\sum_{i = t -w +1}^{t} \Delta P_{i}^{+} }{ \sum_{i = t -w +1}^{t} \Delta P_{i}^{-} }
$$

para $t \in [w + 1, n]$ y en donde
$$
P_{i}^{+} = max(P_{i} - P_{i -1}, 0)
$$

$$
P_{i}^{-} = |min(P_{i} - P_{i -1}, 0)|
$$


El $RSI$, entonces, se define como:

$$
RSI_{t|w}  =
\begin{equation}
\left\{
\begin{aligned}
 100, \quad \text{si}\, \sum_{i = t -w +1}^{t} \Delta P_{i}^{-} = 0 \\
100 - \dfrac{100}{1 + RS_{t|w}} \quad \text{en otro caso} 
\end{aligned} \right.
\end{equation}
$$


Para determinar las señales de compra o venta se establecen dos niveles; uno superior ($LS$) y otro inferior ($LI$).

+ Si $RSI_{t|w} > LI$ y $RSI_{t-1|w} < LI$, entonces **compra en $t$**.

+ Si $RSI_{t|w} < LS$ y $RSI_{t-1|w} > LS$, entonces **venta en $t$**.

## Ejercicios
+ Programe una función para calcular la serie de tiempo del RSI. El resultado debe de ser un numpy array, con la misma longitud de 'datos', conteniendo los valores del indicador. En caso de que no se tenga suficiente información para calcular el valor en un momento $t$, el valor que se debe de asignar es numpy.nan

+ Programe una función que genere las señales de compra y venta para cada periodo $t$. El resultado debe de ser un numpy array de strings que representa la señal para dicho periodo ('compra', 'venta' o 'espera'). Si el valor del indicador, en el periodo $t$, es numpy.nan entonces se asigna 'espera'.

+ Haga una gráfica de la serie de tiempo del indicador RSI junto con los límites $LS$ y $LI$.

In [ ]:
#Escriba aquí su código

## Generando los resultados de una estrategia

Para poder evaluar el desempeño de las estrategias generadas por los indicadores técnicos, es necesario calcular la ganancia/pérdida obtenida al seguir las señales de compra y venta.

Programe una función que calcule la ganancia de una estrategia bajo los siguientes supuestos:

1. Se inicia con un capital de M = $ \$100,000.00$

2. No se permiten ventas en corto, es decir, sólo podemos ejecutar una señal de venta si antes de esta hubo una señal de compra, como consecuencia de este punto, la primera señal no puede ser una señal de venta.

3. Ya que para calcular los indicadores técnicos estamos utilizando precios de cierre ajustados, las señales son generadas al cierre del periodo $t$. Suponga entonces que si en $t$ se genera una señal (de compra o venta) entonces la transacción se ejecuta en el periodo $t + 1$ utilizando como precio de ejecución el promedio entre precio más grande y el más pequeño en $t + 1$.

4. Sólo podemos comprar un número entero de acciones y destinamos todo nuestro capital disponible para comprar tantas acciones sean posibles, es decir, si $t$ es el momento en que se ejecuta una compra, $P_{t}^{ejec}$ es el precio de ejecución y $C_{t}$ nuestro capital disponible en el tiempo $t$, entonces el número de acciones que se compran es:

$$
A_{t} = \Big \lfloor \tfrac{C_{t}} { P_{t}^{ejec} (1 + k)} \Big \rfloor
$$

en donde $\lfloor x \rfloor$ es la función entero, es decir, la parte entera de $x$ y $k$ es el porcentaje de costo de transacción.

5. El costo de transacción es igual a $k = 0.25\%$ sobre el monto de la operación, es decir:

    + En una compra pagamos $P_{t}^{ejec} (1 + k)$.
    + En una venta recibimos $P_{t}^{ejec} (1 - k)$.    
    
6. En el último periodo, se cierran posiciones abiertas (si las hay). Por ejemplo, suponga que en el tiempo $t_1$ se ejecutó una compra y nuestro arreglo que contiene la estrategia, $a$, ya no contiene una venta entonces forzamos a que ```a[-1] = 'venta'``` y esta venta se ejecuta en ese mismo periodo, no en el siguiente periodo.

7. Como consecuencia de los puntos 2 y 4, no se permite ejecutar compras o ventas sucesivas.

### Ejemplo

Suponga que en los periodos $t_{1}$ y $t_{3}$ tenemos señales de compra, mientras que en $t_{2}$ y $t_{4}$ son periodos en los que se reciben señales de venta $t_{1} < t_{2} < t_{3} < t_{4}$. Suponga además que en $t_{1}$ cuenta con un capital de $C_1$.

* **Se ejecuta compra en** $t_{1} + 1$:  

recibimos 
$$
A_{t_{1} + 1} = \Big \lfloor \tfrac{C_{1}} { P_{t_{1} + 1 }^{ejec} (1 + k)} \Big \rfloor
$$
acciones. Y nos sobra un capital de $C_2 = C_1 - P_{t_{1} + 1 }^{ejec} A_{t_{1} + 1} (1 + k)$.

* **Se ejecuta venta en** $t_{2} + 1$:

Vendemos todas las acciones disponibles, $A_{t_{1} + 1}$, y terminamos con un capital de $C_{3} = C_{2} + A_{t_{1} + 1} P_{t_{2} + 1}^{ejec}(1 - k)$

* **Se ejecuta compra en** $t_{3} + 1$:  

recibimos 
$$
A_{t_{3} + 1} = \Big \lfloor \tfrac{C_{3}} { P_{t_{3} + 1 }^{ejec} (1 + k)} \Big \rfloor
$$
acciones. Y nos sobra un capital de $C_4 = C_3 - P_{t_{3} + 1 }^{ejec} A_{t_{3} + 1} (1 + k)$.

* **Se ejecuta venta en** $t_{4} + 1$:

Vendemos todas las acciones disponibles, $A_{t_{3} + 1}$, y terminamos con un capital de $C_{5} = C_{4} + A_{t_{3} + 1} P_{t_{4} + 1}^{ejec}(1 - k)$

Nuestra ganancia porcentual es 
$$
\dfrac{C_{5} - M}{M}.
$$


In [ ]:
#Escriba aquí su código